In [1]:
import torch
import torchvision
import construct
import prep
import pandas as pd

In [2]:
image_dir = 'data/10x'
labels_file = 'data/me.csv'

DATA = prep.prep_data(pd.read_csv(labels_file), image_dir)

transforms = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.RandomRotation((-180,180)),
                            torchvision.transforms.CenterCrop((325)),
                            torchvision.transforms.ToTensor()
                                      ])

/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/prep.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = False
/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/prep.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = True


In [3]:
class myModel(torch.nn.Module):
    def __init__(self):
        """
        Initializes CNN. Here we just define layer shapes that we call in the forward func
        """
        super().__init__()

        self.conv1 = torch.nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5)
                
        #Convultion layer 2. See above
        self.conv2 = torch.nn.Conv2d(in_channels = 6, 
                               out_channels = 12, 
                               kernel_size = 5)
        
        self.fc_1 = torch.nn.Linear(39 * 39 * 12, 256)
        self.fc_2 = torch.nn.Linear(256, 2)
        self.drop = torch.nn.Dropout(p=.2)
        self.batch1 = torch.nn.BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch2 = torch.nn.BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            
    def forward(self, x):
        """
        Function that performs all the neural network forward calculation i.e.
        takes image data from the input of the neural network to the output
        """
        
        x = self.conv1(x)
        x = self.batch1(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 2)
        x = torch.nn.functional.leaky_relu(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 4)
        x = torch.nn.functional.leaky_relu(x)
        x = x.view(x.shape[0], -1)  
        x = self.fc_1(x) 
        x = torch.nn.functional.leaky_relu(x)
        x = self.drop(x)
        x = self.fc_2(x) 
        return x

model = myModel()

In [4]:
#Declaring iterator. The thing that will loop through our dataset.

data = prep.tenX_dataset(DATA, 'data/10x', transform=transforms)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test, train = torch.utils.data.random_split(data, [int(.15*len(data)), len(data)-int(.15*len(data))])
BATCH_SIZE = 32
epochs = 150

In [5]:
cnn = construct.train(epochs, BATCH_SIZE, train, criterion, 'Adam', model)

EPOCH: 0, acc: 0.7495039701461792, loss: 5.826170656416151
tensor([[  1.3418,   2.0230],
        [ 15.1528, -26.2446],
        [  7.7074,  -9.8131],
        [ -2.4488,   1.5040],
        [  4.4918,  -2.1442],
        [ 10.4643, -14.4138],
        [  0.9801,   0.8238]], grad_fn=<AddmmBackward>)
tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])
EPOCH: 1, acc: 0.7847222222222222, loss: 0.6980671903325452
EPOCH: 2, acc: 0.8015873034795126, loss: 0.7744539562198851
EPOCH: 3, acc: 0.8576388888888888, loss: 0.3752673765023549
EPOCH: 4, acc: 0.848710318406423, loss: 0.39321037630240124
EPOCH: 5, acc: 0.8715277777777778, loss: 0.31987442407343125
tensor([[  1.7943, -18.0507],
        [  0.5911, -13.1254],
        [  3.2723,  -0.5083],
        [  1.0217,  -0.7675],
        [ -0.2604,   0.7032],
        [ -0.3351,  -3.2899],
        [  0.5031,   1.2254]], grad_fn=<AddmmBackward>)
tensor([[1, 0],
        [1, 0],
        [1, 0],
       

EPOCH: 62, acc: 0.9201388888888888, loss: 0.22569085326459673
EPOCH: 63, acc: 0.8849206368128458, loss: 0.23806856241491106
EPOCH: 64, acc: 0.911210318406423, loss: 0.2530073614584075
EPOCH: 65, acc: 0.8958333333333334, loss: 0.23456614795658323
tensor([[-3.9419e-01,  3.8355e+00],
        [ 1.1219e+00, -1.1340e+00],
        [ 7.2946e+00, -7.3998e+00],
        [-4.2779e-01, -1.5810e+00],
        [ 4.9212e+00, -8.6220e+00],
        [ 2.7718e+00, -3.4016e+00],
        [ 2.1463e-01,  8.1265e-03]], grad_fn=<AddmmBackward>)
tensor([[0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])
EPOCH: 66, acc: 0.9201388888888888, loss: 0.19267985224723816
EPOCH: 67, acc: 0.9305555555555556, loss: 0.255658561984698
EPOCH: 68, acc: 0.9042658739619784, loss: 0.23440636694431305
EPOCH: 69, acc: 0.8938492072953118, loss: 0.2546912812524372
EPOCH: 70, acc: 0.8903769850730896, loss: 0.2524317767884996
tensor([[ 2.3482, -3.8019],
        [ 4.0008, -3.4400],
 

EPOCH: 126, acc: 0.870535714758767, loss: 0.4160245441728168
EPOCH: 127, acc: 0.9042658739619784, loss: 0.21738829712073007
EPOCH: 128, acc: 0.8988095257017348, loss: 0.2286173320478863
EPOCH: 129, acc: 0.9002976218859354, loss: 0.3336373832490709
EPOCH: 130, acc: 0.9077380961842008, loss: 0.2618563166922993
tensor([[ 2.4489, -1.4530],
        [ 3.6335, -3.1248],
        [ 3.0651, -2.5491],
        [ 0.1629, -0.3140],
        [ 0.2568, -0.9927],
        [ 1.2122, -0.6772],
        [ 6.8082, -2.7440]], grad_fn=<AddmmBackward>)
tensor([[1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0]])
EPOCH: 131, acc: 0.8958333333333334, loss: 0.2352118074066109
EPOCH: 132, acc: 0.9181547628508674, loss: 0.19967858658896553
EPOCH: 133, acc: 0.9201388888888888, loss: 0.20953111598889032
EPOCH: 134, acc: 0.9077380961842008, loss: 0.21970788058307436
EPOCH: 135, acc: 0.9375, loss: 0.2250921974579493
tensor([[ 3.1057, -2.6260],
        [-0.0510, -0.0552]

In [10]:
images, labels, predictions, weights, acc = construct.get_predictions(BATCH_SIZE, cnn, test)

In [11]:
(labels[:,1] == predictions).float().sum()/len(predictions)

tensor(0.8478)

In [12]:
(labels[:,1] == 0).float().sum()/len(predictions)

tensor(0.6957)

In [9]:
def test_dataset_class():
    filename = '../tests/test_assets/test_labels_unclean.csv'
    image_dir = '../tests/test_assets/test_images'
    
    #Eventually just read in already cleaned labels sheet
    labels = prep_data(pd.read_csv(filename), image_dir)
    
    
    transform = None
    test_dataset = tenX_dataset(labels, image_dir, transform)
    
    #len() check
    length = len(test_dataset)
    expect = 5
    assert length is expect, f'10x dataset length method failed. Got {length}, should be {expect}'
    
    #get_item() check
    samples = []
    keys = ['image', 'plastic','shape','color']
    for i in range(len(test_dataset)):
        assert test_dataset[i]['image'] is not None, 'Got NoneType instead of image'
        isP = test_dataset[i]['plastic']
        assert math.isclose(0,isP) or math.isclose(1,isP), f'plastic not 0 or 1, instead is {isP}'
        length = len(test_dataset[i]['shape'])
        assert length is 4, f'length of shape array not 4, instead is {length}'
        samples.append(test_dataset[i])
        
    shape = samples[0]['shape']
    assert math.isclose(shape[2], 1),  f'wrong shape first image, is {shape}'
    color = samples[0]['color']
    assert math.isclose(color[0], 1), f'wrong color first image, is {color}'
    assert math.isclose(samples[4]['plastic'], 1), 'wrong plastic id 3rd image'
    
test_dataset_class()

NameError: name 'prep_data' is not defined

In [ ]:
#0 - BASE LINE 82% naive and 88% model test set accuracy
#1 - 9 output CNN1, 18 output CNN2 87% test vs 77% naive
#2 - 3rd linear layer telescoped (12*39*39 -> 2048 -> 256- >2). 93% test, 87% naive
#3 - added a convulutional layer, 12 -> 24 no good
#2d dropout after convolutions led to predicting all pastics, i also read dropouts after convultions is bad, going to try batch normalizations though
#Another paper I read said that non-adaptive optimizers were better for CNN's if thehyperparameters are tunned
#tuned correctly but are much slower so I don't want to even try given it takes me 20min to train already
#adding batchnormalization after convultions improved training results alot, but not test which is weird since
#its supposed to reduce overfitting. Im going to run again overnight